# Análisis Exploratorio de Datos (EDA) - Detección de Fraude Financiero

Este notebook contiene el análisis exploratorio completo del dataset de fraude financiero.

**Objetivos:**
- Comprender la estructura y distribución de los datos
- Identificar patrones y relaciones entre variables
- Detectar anomalías y valores atípicos
- Definir reglas de validación de datos
- Proponer features derivados

## 1. Carga e Importación de Librerías

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import warnings

# Configuración
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Importar módulo de carga de datos
from mlops_pipeline.src.cargar_datos import DataLoader

print("✓ Librerías importadas correctamente")

## 2. Carga de Datos

In [ ]:
# Cargar datos usando DataLoader
loader = DataLoader()
df = loader.load_data()

print(f"\nShape del dataset: {df.shape}")
print(f"Número de filas: {df.shape[0]:,}")
print(f"Número de columnas: {df.shape[1]}")

## 3. Exploración Inicial

In [ ]:
# Primeras filas del dataset
print("\n📊 PRIMERAS 5 FILAS DEL DATASET:")
df.head()

In [ ]:
# Información general del dataset
print("\n📋 INFORMACIÓN DEL DATASET:")
df.info()

In [ ]:
# Estadísticas descriptivas de variables numéricas
print("\n📈 ESTADÍSTICAS DESCRIPTIVAS (Variables Numéricas):")
df.describe()

In [ ]:
# Tipos de datos
print("\n🔍 TIPOS DE DATOS:")
print(df.dtypes)

# Clasificación de variables
numerical_vars = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_vars = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nVariables numéricas ({len(numerical_vars)}): {numerical_vars}")
print(f"Variables categóricas ({len(categorical_vars)}): {categorical_vars}")

In [ ]:
# Verificar valores nulos
print("\n🔎 VALORES NULOS:")
null_counts = df.isnull().sum()
null_percentage = (null_counts / len(df)) * 100

null_df = pd.DataFrame({
    'Cantidad': null_counts,
    'Porcentaje': null_percentage
})

print(null_df[null_df['Cantidad'] > 0])

if null_counts.sum() == 0:
    print("\n✓ No se encontraron valores nulos en el dataset")

In [ ]:
# Verificar duplicados
duplicates = df.duplicated().sum()
print(f"\n🔄 FILAS DUPLICADAS: {duplicates}")

if duplicates == 0:
    print("✓ No se encontraron filas duplicadas")

## 4. Análisis Univariable

### 4.1 Variable Objetivo (isFraud)

In [ ]:
# Distribución de la variable objetivo
fraud_counts = df['isFraud'].value_counts()
fraud_pct = df['isFraud'].value_counts(normalize=True) * 100

print("\n🎯 DISTRIBUCIÓN DE LA VARIABLE OBJETIVO (isFraud):")
print(f"\nNo Fraude (0): {fraud_counts[0]:,} ({fraud_pct[0]:.2f}%)")
print(f"Fraude (1):    {fraud_counts[1]:,} ({fraud_pct[1]:.2f}%)")
print(f"\nRatio de desbalanceo: 1:{fraud_counts[0]/fraud_counts[1]:.0f}")

# Gráfico
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Countplot
sns.countplot(data=df, x='isFraud', ax=axes[0], palette='Set2')
axes[0].set_title('Distribución de Transacciones Fraudulentas', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Fraude (0=No, 1=Sí)', fontsize=12)
axes[0].set_ylabel('Cantidad', fontsize=12)

# Pie chart
axes[1].pie(fraud_counts, labels=['No Fraude', 'Fraude'], autopct='%1.2f%%', 
           colors=['#66b3ff', '#ff9999'], startangle=90)
axes[1].set_title('Proporción de Fraude vs No Fraude', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ OBSERVACIÓN: Dataset altamente desbalanceado - se requiere técnica de balanceo")

### 4.2 Variables Numéricas

In [ ]:
# Análisis estadístico de variables numéricas
numeric_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrg']

print("\n📊 ESTADÍSTICAS DESCRIPTIVAS DETALLADAS:")
print("="*80)

for col in numeric_cols:
    print(f"\n{col.upper()}:")
    print(f"  Media:               {df[col].mean():,.2f}")
    print(f"  Mediana:             {df[col].median():,.2f}")
    print(f"  Moda:                {df[col].mode()[0]:,.2f}")
    print(f"  Desviación Estándar: {df[col].std():,.2f}")
    print(f"  Varianza:            {df[col].var():,.2f}")
    print(f"  Rango:               {df[col].min():,.2f} - {df[col].max():,.2f}")
    print(f"  IQR (Q3-Q1):         {df[col].quantile(0.75) - df[col].quantile(0.25):,.2f}")
    print(f"  Skewness (Asimetría):{df[col].skew():,.4f}")
    print(f"  Kurtosis:            {df[col].kurtosis():,.4f}")

In [ ]:
# Histogramas de variables numéricas
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(numeric_cols):
    # Histograma
    axes[idx].hist(df[col], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribución de {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Frecuencia', fontsize=10)
    axes[idx].grid(alpha=0.3)
    
    # Histograma con escala log
    axes[idx + 3].hist(df[col][df[col] > 0], bins=50, color='coral', edgecolor='black', alpha=0.7)
    axes[idx + 3].set_title(f'Distribución de {col} (Escala Log)', fontsize=12, fontweight='bold')
    axes[idx + 3].set_xlabel(col, fontsize=10)
    axes[idx + 3].set_ylabel('Frecuencia', fontsize=10)
    axes[idx + 3].set_yscale('log')
    axes[idx + 3].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots de variables numéricas
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, col in enumerate(numeric_cols):
    sns.boxplot(data=df, y=col, ax=axes[idx], color='lightgreen')
    axes[idx].set_title(f'Boxplot de {col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(col, fontsize=10)

plt.tight_layout()
plt.show()

print("\n⚠️ OBSERVACIÓN: Presencia significativa de outliers en todas las variables numéricas")

### 4.3 Variables Categóricas

In [ ]:
# Análisis de la variable 'type'
print("\n💳 DISTRIBUCIÓN DE TIPO DE TRANSACCIÓN:")
print("="*80)

type_counts = df['type'].value_counts()
type_pct = df['type'].value_counts(normalize=True) * 100

type_df = pd.DataFrame({
    'Cantidad': type_counts,
    'Porcentaje': type_pct
})

print(type_df)

# Gráficos
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Countplot
sns.countplot(data=df, x='type', ax=axes[0], palette='viridis', order=type_counts.index)
axes[0].set_title('Distribución de Tipos de Transacción', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Tipo de Transacción', fontsize=12)
axes[0].set_ylabel('Cantidad', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)

# Barplot horizontal con porcentajes
type_pct.plot(kind='barh', ax=axes[1], color='teal')
axes[1].set_title('Porcentaje de Tipos de Transacción', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Porcentaje (%)', fontsize=12)
axes[1].set_ylabel('Tipo de Transacción', fontsize=12)

plt.tight_layout()
plt.show()

## 5. Análisis Bivariable y Multivariable

### 5.1 Relación entre Variables Numéricas

In [ ]:
# Matriz de correlación
print("\n🔗 MATRIZ DE CORRELACIÓN:")
correlation_matrix = df[numeric_cols + ['isFraud']].corr()
print(correlation_matrix)

# Heatmap de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Variables Numéricas y Target', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Pairplot (con muestra para mejor rendimiento)
print("\n📊 GENERANDO PAIRPLOT (muestra de 5,000 registros)...")

# Tomar una muestra estratificada
df_sample = df.groupby('isFraud', group_keys=False).apply(lambda x: x.sample(min(len(x), 2500)))

pairplot_vars = numeric_cols + ['isFraud']
sns.pairplot(df_sample[pairplot_vars], hue='isFraud', palette='Set1', 
            diag_kind='kde', plot_kws={'alpha': 0.6}, height=2.5)
plt.suptitle('Pairplot - Variables Numéricas vs Fraude', y=1.02, fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### 5.2 Relación entre Variables Numéricas y Fraude

In [ ]:
# Boxplots de variables numéricas vs Fraude
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, col in enumerate(numeric_cols):
    sns.boxplot(data=df, x='isFraud', y=col, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{col} vs Fraude', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Fraude (0=No, 1=Sí)', fontsize=10)
    axes[idx].set_ylabel(col, fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Distribuciones comparativas
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, col in enumerate(numeric_cols):
    df[df['isFraud'] == 0][col].hist(bins=50, alpha=0.6, label='No Fraude', 
                                      ax=axes[idx], color='blue', density=True)
    df[df['isFraud'] == 1][col].hist(bins=50, alpha=0.6, label='Fraude', 
                                      ax=axes[idx], color='red', density=True)
    axes[idx].set_title(f'Distribución de {col} por Fraude', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Densidad', fontsize=10)
    axes[idx].legend()

plt.tight_layout()
plt.show()

### 5.3 Relación entre Variables Categóricas y Fraude

In [ ]:
# Tabla cruzada: type vs isFraud
print("\n📋 TABLA CRUZADA: Tipo de Transacción vs Fraude")
print("="*80)

crosstab = pd.crosstab(df['type'], df['isFraud'], margins=True)
print(crosstab)

# Tabla con porcentajes
print("\n📊 PORCENTAJES (por tipo de transacción):")
crosstab_pct = pd.crosstab(df['type'], df['isFraud'], normalize='index') * 100
print(crosstab_pct)

# Gráfico
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Countplot agrupado
sns.countplot(data=df, x='type', hue='isFraud', ax=axes[0], palette='Set1')
axes[0].set_title('Tipo de Transacción vs Fraude (Conteo)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Tipo de Transacción', fontsize=12)
axes[0].set_ylabel('Cantidad', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title='Fraude', labels=['No', 'Sí'])

# Heatmap de tabla cruzada
sns.heatmap(crosstab_pct, annot=True, fmt='.2f', cmap='YlOrRd', ax=axes[1], 
            cbar_kws={'label': 'Porcentaje (%)'})
axes[1].set_title('Tipo de Transacción vs Fraude (% por Tipo)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Fraude', fontsize=12)
axes[1].set_ylabel('Tipo de Transacción', fontsize=12)

plt.tight_layout()
plt.show()

print("\n⚠️ HALLAZGO CLAVE: Ciertos tipos de transacción tienen mayor propensión al fraude")

## 6. Análisis de Outliers

In [ ]:
# Detección de outliers usando IQR
print("\n🔍 DETECCIÓN DE OUTLIERS (Método IQR):")
print("="*80)

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_pct = (len(outliers) / len(df)) * 100
    
    print(f"\n{col}:")
    print(f"  Límite inferior: {lower_bound:,.2f}")
    print(f"  Límite superior: {upper_bound:,.2f}")
    print(f"  Outliers:        {len(outliers):,} ({outlier_pct:.2f}%)")

## 7. Conclusiones y Recomendaciones

### 7.1 Hallazgos Principales

**Hallazgos del Análisis Exploratorio:**

1. **Desbalanceo de Clases:**
   - El dataset está altamente desbalanceado (~99% no fraude vs ~1% fraude)
   - **Acción:** Aplicar técnicas de balanceo (RandomUnderSampler, SMOTE, class_weight)

2. **Calidad de Datos:**
   - No se encontraron valores nulos
   - No se encontraron duplicados
   - Todos los tipos de datos son consistentes

3. **Variables Numéricas:**
   - Distribuciones altamente asimétricas (skewed)
   - Presencia significativa de outliers (valores extremos)
   - **Acción:** Aplicar normalización/estandarización

4. **Tipo de Transacción:**
   - CASH_OUT y TRANSFER tienen mayor incidencia de fraude
   - PAYMENT rara vez es fraudulento
   - **Acción:** Feature importante para el modelo

5. **Correlaciones:**
   - Baja correlación entre variables numéricas y fraude
   - **Acción:** Crear features derivados más informativos

### 7.2 Reglas de Validación de Datos

**Reglas de Validación Identificadas:**

1. **Regla 1:** `amount` no puede ser negativo
   - Validación: `amount >= 0`

2. **Regla 2:** `type` solo puede contener valores específicos
   - Valores permitidos: ['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']

3. **Regla 3:** `isFraud` debe ser binario
   - Valores permitidos: [0, 1]

4. **Regla 4:** Los balances no pueden ser negativos
   - Validación: `oldbalanceOrg >= 0` y `newbalanceOrg >= 0`

5. **Regla 5:** Columnas irrelevantes deben eliminarse
   - Eliminar: ['step', 'nameOrig', 'nameDest']

### 7.3 Features Derivados Sugeridos

**Features Propuestos para Ingeniería de Características:**

1. **errorBalanceOrg:**
   - Fórmula: `oldbalanceOrg - newbalanceOrg - amount`
   - Detecta inconsistencias en el balance después de la transacción

2. **transactionRatio:**
   - Fórmula: `amount / (oldbalanceOrg + 1)`
   - Indica qué proporción del balance representa la transacción

3. **zeroBalanceAfter:**
   - Fórmula: `1 if newbalanceOrg == 0 else 0`
   - Marca transacciones que vacían completamente la cuenta

4. **isHighAmount:**
   - Fórmula: `1 if amount > percentile_95 else 0`
   - Identifica transacciones de monto inusualmente alto

Estos features capturan patrones sospechosos que pueden indicar fraude.

## 8. Próximos Pasos

1. **Validación de Datos:**
   - Implementar las reglas de validación en `data_validation.py`

2. **Ingeniería de Características:**
   - Crear los features derivados en `ft_engineering.py`
   - Aplicar transformaciones (StandardScaler, OneHotEncoder)

3. **Modelado:**
   - Entrenar múltiples modelos (LogisticRegression, RandomForest, XGBoost)
   - Aplicar técnicas de balanceo
   - Optimizar hiperparámetros

4. **Evaluación:**
   - Usar métricas apropiadas para datos desbalanceados (ROC-AUC, F1, Recall)
   - Analizar matriz de confusión

5. **Despliegue:**
   - Crear API con FastAPI
   - Implementar monitoreo de drift